### Creating functions and Example Usage

In [1]:
from functions import tools, available_functions
from dotenv import load_dotenv ,find_dotenv
from openai import OpenAI
import time
import json
import os

In [2]:
%pip install --upgrade openai

Note: you may need to restart the kernel to use updated packages.


In [3]:
def initialize_openai_client():
    _ : bool = load_dotenv(find_dotenv("E:/Python/.env"))
    client : OpenAI = OpenAI()
    return client  

client = initialize_openai_client()



def show_json(obj):
    print(json.dumps(json.loads(obj.model_dump_json()), indent=4))



In [4]:
assistant = client.beta.assistants.create(
    name = "Financial Analyst",
    instructions = "Act as a financial analyst by accessing detailed financial data through the Financial Modeling Prep API. Your capabilities include analyzing key metrics, comprehensive financial statements, vital financial ratios, and tracking financial growth trends. ",
    tools = tools,
    model = "gpt-3.5-turbo-1106"
)

# show_json(assistant)

In [5]:
thread = client.beta.threads.create()

In [6]:
print(thread)

Thread(id='thread_Y7Wzj59Ss7Qk3tL2IagPnunO', created_at=1704579810, metadata={}, object='thread')


In [1]:
print(type(thread))

NameError: name 'thread' is not defined

In [7]:
def submit_message(assistant_id, thread, user_message):
    """
    Submit a message to the Financial Analyst.

    :param assistant_id: The ID of the Financial Analyst.
    :param thread: The ID of the thread to send the message to.
    :param user_message: The message to send.
    :return: The response from the Financial Analyst.
    """
    client.beta.threads.messages.create(
        thread_id = thread.id,
        role = "user",
        content = user_message)
    run = client.beta.threads.runs.create(
        assistant_id = assistant_id,
        thread_id = thread.id
    )
    return run

In [8]:
def wait_on_run(run, thread, available_functions):
    while True:
        run = client.beta.threads.runs.retrieve(thread_id=thread.id, run_id=run.id)

        if run.status == "requires_action" and run.required_action is not None:
            tool_calls = run.required_action.submit_tool_outputs.tool_calls
            tool_outputs = []

            for tool_call in tool_calls:
                function_name = tool_call.function.name
                function_args = json.loads(tool_call.function.arguments)

                if function_name in available_functions:
                    function_to_call = available_functions[function_name]
                    
                    # Move the output definition here
                    output = function_to_call(**function_args)
                    
                    tool_outputs.append({
                        "tool_call_id": tool_call.id,
                        "output": output,
                    })

            # Submit tool outputs and update the run
            client.beta.threads.runs.submit_tool_outputs(
                thread_id=thread.id,
                run_id=run.id,
                tool_outputs=tool_outputs
            )

        elif run.status == "completed":
            print("Run Completed")
            break
            # # List the messages to get the response
            # messages = client.beta.threads.messages.list(thread_id=thread.id)
            # response = ""
            # for message in messages.data:
            #     role_label = "User" if message.role == "user" else "Assistant"
            #     message_content = message.content[0].text.value
            #     response += f"{role_label}: {message_content}\n"

            # # print(response)  # Print the response after processing the completed run
            # return response  # Return the response after processing the completed run
            # break

        elif run.status == "failed":
            print("Run Failed.")
            break

        elif run.status in ["in_progress", "queued"]:
            print(f"Run is {run.status}. Waiting...")
            time.sleep(5)  # Wait for 5 seconds before checking again

        else:
            print(f"Unexpected status: {run.status}")
            break
    return run


In [9]:
def get_response(thread):
    response = client.beta.threads.messages.list(
        threadId=thread,
        order = "asc"
    )
    if response:
        return response
    else:
        return "Unable to get Response from Assistant"

In [10]:
def pretty_print(messages):
    responses = []
    for message in messages:
        if message.role == "user":
            responses.append("You: " + message.content[0].text.value)
        elif message.role == "assistant":
            responses.append("Bot: " + message.content[0].text.value)
            
    return "\n".join(responses)

### Code without Doc Strings, Type Hints

In [ ]:
from functions import tools, available_functions
from dotenv import load_dotenv ,find_dotenv
from openai import OpenAI
import time
import json
import os

def initialize_openai_client():
    load_dotenv(find_dotenv("E:/Python/.env"))
    openai_key = os.environ.get("OPENAI_API_KEY")
    client = OpenAI()
    return client  

def show_json(obj):
    print(json.dumps(json.loads(obj.model_dump_json()), indent=4))

def submit_message(assistant_id, thread, user_message):
    """
    Submit a message to the Financial Analyst.

    :param assistant_id: The ID of the Financial Analyst.
    :param thread: The ID of the thread to send the message to.
    :param user_message: The message to send.
    :return: The response from the Financial Analyst.
    """
    client.beta.threads.messages.create(
        thread_id = thread.id,
        role = "user",
        content = user_message)
    run = client.beta.threads.runs.create(
        assistant_id = assistant_id,
        thread_id = thread.id
    )
    return run

def wait_on_run(run, thread, available_functions):
    while True:
        run = client.beta.threads.runs.retrieve(thread_id=thread.id, run_id=run.id)

        if run.status == "requires_action" and run.required_action is not None:
            tool_calls = run.required_action.submit_tool_outputs.tool_calls
            tool_outputs = []

            for tool_call in tool_calls:
                function_name = tool_call.function.name
                function_args = json.loads(tool_call.function.arguments)

                if function_name in available_functions:
                    function_to_call = available_functions[function_name]
                    
                    # Move the output definition here
                    output = function_to_call(**function_args)
                    
                    tool_outputs.append({
                        "tool_call_id": tool_call.id,
                        "output": output,
                    })

            # Submit tool outputs and update the run
            client.beta.threads.runs.submit_tool_outputs(
                thread_id=thread.id,
                run_id=run.id,
                tool_outputs=tool_outputs
            )

        elif run.status == "completed":
            print("Run Completed")
            break
            # # List the messages to get the response
            # messages = client.beta.threads.messages.list(thread_id=thread.id)
            # response = ""
            # for message in messages.data:
            #     role_label = "User" if message.role == "user" else "Assistant"
            #     message_content = message.content[0].text.value
            #     response += f"{role_label}: {message_content}\n"

            # # print(response)  # Print the response after processing the completed run
            # return response  # Return the response after processing the completed run
            # break

        elif run.status == "failed":
            print("Run Failed.")
            break

        elif run.status in ["in_progress", "queued"]:
            print(f"Run is {run.status}. Waiting...")
            time.sleep(5)  # Wait for 5 seconds before checking again

        else:
            print(f"Unexpected status: {run.status}")
            break
    return run

def get_response(thread):
    response = client.beta.threads.messages.list(
        threadId=thread,
        order = "asc"
    )
    if response:
        return response
    else:
        return "Unable to get Response from Assistant"
    
def pretty_print(messages):
    responses = []
    for message in messages:
        if message.role == "user":
            responses.append("You: " + message.content[0].text.value)
        elif message.role == "assistant":
            responses.append("Bot: " + message.content[0].text.value)
            
    return "\n".join(responses)


client = initialize_openai_client()

assistant = client.beta.assistants.create(
    name = "Financial Analyst",
    instructions = "Act as a financial analyst by accessing detailed financial data through the Financial Modeling Prep API. Your capabilities include analyzing key metrics, comprehensive financial statements, vital financial ratios, and tracking financial growth trends. ",
    tools = tools,
    model = "gpt-3.5-turbo-1106"
)

thread = client.beta.threads.create()